<a href="https://colab.research.google.com/github/yunji-1ee/GitTeamProject/blob/main/YJ/DLprject_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM

In [12]:
pip install tensorflow

In [13]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

#Warnings 제거
import warnings
warnings.filterwarnings('ignore')
#구글 마운트
from google.colab import drive
drive.mount('/content/drive')
#경로설정
import os
os.chdir("/content/drive/MyDrive/DL")
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/DL'

## **데이터 살펴보기**


|Column|Description|
|:---|:---|
|stock_id|주식의 고유 식별자|
|date_id|날짜 고유 식별자|
|seconds_in_bucket|당일 마감경매시작이후 경과된 초|
|imbalance_size|참조가격에서 불균형한 금액|
|imbalance_buy_sell_flag|불균형의 종류|
|reference_price|최대 수익 매수매도를 위한 가격|
|matched_size|기준 가격으로 거래가능한 금액의 크기|
|far_price|거래만볼 떄 매물을 최대로 살 수 있는 가격|
|near_price|지속적 거래로 매물을 최대로 살 수 있는 가격|

.
.
등등,,
```
```

In [16]:
# 데이터 불러오기
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')
rt_df = pd.read_csv('revealed_targets.csv')

In [15]:
train_df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 17 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   stock_id                 1048575 non-null  int64  
 1   date_id                  1048575 non-null  int64  
 2   seconds_in_bucket        1048575 non-null  int64  
 3   imbalance_size           1048520 non-null  float64
 4   imbalance_buy_sell_flag  1048575 non-null  int64  
 5   reference_price          1048520 non-null  float64
 6   matched_size             1048520 non-null  float64
 7   far_price                467152 non-null   float64
 8   near_price               475250 non-null   float64
 9   bid_price                1048520 non-null  float64
 10  bid_size                 1048575 non-null  float64
 11  ask_price                1048520 non-null  float64
 12  ask_size                 1048575 non-null  float64
 13  wap                      1048520 non-null 

In [18]:
#결측치 확인
print(train_df.isnull().sum())

stock_id                        0
date_id                         0
seconds_in_bucket               0
imbalance_size                 55
imbalance_buy_sell_flag         0
reference_price                55
matched_size                   55
far_price                  581423
near_price                 573325
bid_price                      55
bid_size                        0
ask_price                      55
ask_size                        0
wap                            55
target                         31
time_id                         0
row_id                          0
dtype: int64


In [19]:
#train_df 데이터의 far_price와 near_price의 결측치에 0으로 채워넣기
train_df['far_price'].fillna(0, inplace=True)
train_df['near_price'].fillna(0, inplace=True)

In [20]:
print(train_df.isnull().sum())

stock_id                    0
date_id                     0
seconds_in_bucket           0
imbalance_size             55
imbalance_buy_sell_flag     0
reference_price            55
matched_size               55
far_price                   0
near_price                  0
bid_price                  55
bid_size                    0
ask_price                  55
ask_size                    0
wap                        55
target                     31
time_id                     0
row_id                      0
dtype: int64
